# Data in a day

## Import libraries

In [ ]:
# 🐼 is to work with tables of data (http://pandas.pydata.org/)
import pandas as pd

# sklearn is for machine learning (http://scikit-learn.org)
from sklearn import tree

# matplotlib is to make plots, pandas using it under the hood
# Display plots in this page rather than open another page
%matplotlib inline

import seaborn as sns

import graphviz 

from sklearn.model_selection import cross_val_score, GridSearchCV

## Source the data

In [ ]:
df = pd.read_csv('../data-sets/mushrooms.csv')

## Explore and transform the data

In [ ]:
df.head()

In [ ]:
df.describe()

Let's try and visualise this data with the help of https://www.kaggle.com/surajit346/ml-models-and-visualizations-for-beginners

In [ ]:
sns.countplot(x='odor',hue='class',data=df)

### Dealing with categories

Text data needs to be further transformed in order to use machine learning - ie. we need to turn it into numbers.

e.g. odor, we need to treat each type (e.g. odor=n, odor=a) as its own feature to avoid creating ordering in the data where none exists

This is called **one hot** encoding

In [ ]:
features = pd.get_dummies(df)
features.head()

## Building a model

We don't want to include the thing we want to predict as the input data, so lets drop it. Also let's put the classes into their own variable for convenience

In [ ]:
features = features.drop(['class_e','class_p'],axis=1)
classes = df['class']

In [ ]:
# http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier
model = tree.DecisionTreeClassifier()
model.fit(features,classes)

## Evaluate your model

Decision trees are great because you can visualise them, there is a lot of code which we are going to reuse so let's make a function for it

In [1]:
# http://scikit-learn.org/stable/modules/tree.html#classification
def plotTree():
    dot_data = tree.export_graphviz(model, 
                                    out_file=None, 
                                    feature_names=features.columns,
                                    filled=True, 
                                    rounded=True,  
                                    class_names=model.classes_,
                                    special_characters=True)
    graph = graphviz.Source(dot_data)
    return graph 

In [ ]:
plotTree()

It is unfortuante that we can't smell a picture! Also there are lots of things we won't be able to tell from a picture. Can we predict without them?

In [ ]:
features = pd.get_dummies(df.drop(['class','odor','gill-attachment',
                                  'gill-spacing','stalk-shape','stalk-root','stalk-surface-above-ring',
                                 'stalk-surface-below-ring','population','gill-size','habitat','bruises','spore-print-color'],axis=1))
model = tree.DecisionTreeClassifier()
model.fit(features,classes)
plotTree()

Wow, this is a massive tree! Why is it so big? Decision trees in in python are designed to create as many leaves so as to fit the data perfectly. Let's look at the proportion of predictions we got right


In [ ]:
model.score(features,classes)

This is really good, but too good usually means overfitting.

### Avoiding overfitting

So see if our model has overfit the dat we need to split the data into test and train.

Split the data into 3 pieces, 2 are used to train our model, 1 is used to test. Let's see how well the 3 models do when we don't limit the tree depth

In [ ]:
# http://scikit-learn.org/stable/modules/cross_validation.html
train_test = cross_val_score(model, features, classes)
print(train_test)

Averaging to get an overall score when we don't limit the depth of the tree

In [ ]:
train_test.mean()

The smaller average score suggests we did overfit the data. This is because our tree is so complicated because it is really deep. We can limit the depth of the tree and also find the best depth by using grid search

In [ ]:
parameters = {'max_depth':range(2,20)}

modelSearch = GridSearchCV(tree.DecisionTreeClassifier(), parameters, n_jobs=4)

modelSearch.fit(features, classes)

tree_model = modelSearch.best_estimator_

print(modelSearch.best_score_, modelSearch.best_params_) 

model = tree.DecisionTreeClassifier(max_depth=modelSearch.best_params_["max_depth"])
model.fit(features,classes)
plotTree()